In [2]:
import numpy as np
import jax
import jax.numpy as jnp
from tqdm import tqdm
import os
from lib.models.forward_model import UniformForward
from flax import linen as nn
from lib.utils import utils

In [5]:
jnp.arange(10, dtype=jnp.int32)
np.expand_dims(jnp.arange(10, dtype=jnp.int32), 0)

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=int32)

In [ ]:
rate_const = 1
S = 4
B = 2
D = 4
uni = UniformForward(S, rate_const)

rng = jax.random.PRNGKey(1008)
t_rng, sample_rng = jax.random.split(rng)
t = jax.random.uniform(t_rng, (B,))
print(t)
xt = jax.random.randint(sample_rng, shape=(B, D), minval=0, maxval=S, dtype=jnp.int32)
qt0 = uni.transition(t)
qt0_y2x = jnp.transpose(qt0, (0, 2, 1))
print(qt0, qt0.shape)
print(qt0_y2x, qt0_y2x.shape)
print(qt0 == qt0_y2x)

b = jnp.expand_dims(jnp.arange(xt.shape[0]), tuple(range(1, xt.ndim)))


In [10]:
t = jax.random.uniform(t_rng, (B,))
xt = jax.random.randint(sample_rng, shape=(B, D), minval=0, maxval=S, dtype=jnp.int32)
t_eps = t - 0.01
q_teps_0 = uni.transition(t_eps)
print(q_teps_0, q_teps_0.shape)
q_teps_0 = jnp.expand_dims(q_teps_0, axis=list(range(1, xt.ndim)))
print(q_teps_0, q_teps_0.shape)
q_t_teps = uni.transit_between(t_eps, t)
print(q_t_teps, q_t_teps.shape)
q_t_teps = jnp.transpose(q_t_teps, (0, 2, 1))
print(q_t_teps, q_t_teps.shape)



[[[0.27067754 0.2431075  0.2431075  0.2431075 ]
  [0.2431075  0.2706775  0.2431075  0.2431075 ]
  [0.2431075  0.2431075  0.27067754 0.2431075 ]
  [0.2431075  0.2431075  0.2431075  0.2706775 ]]

 [[0.27584603 0.24138466 0.24138464 0.24138466]
  [0.24138466 0.27584606 0.24138466 0.24138466]
  [0.24138464 0.24138466 0.27584603 0.24138466]
  [0.24138466 0.24138466 0.24138466 0.27584606]]] (2, 4, 4)
[[[[0.27067754 0.2431075  0.2431075  0.2431075 ]
   [0.2431075  0.2706775  0.2431075  0.2431075 ]
   [0.2431075  0.2431075  0.27067754 0.2431075 ]
   [0.2431075  0.2431075  0.2431075  0.2706775 ]]]


 [[[0.27584603 0.24138466 0.24138464 0.24138466]
   [0.24138466 0.27584606 0.24138466 0.24138466]
   [0.24138464 0.24138466 0.27584603 0.24138466]
   [0.24138466 0.24138466 0.24138466 0.27584606]]]] (2, 1, 4, 4)
[[[0.97059214 0.00980262 0.00980264 0.00980261]
  [0.00980263 0.9705922  0.00980261 0.00980261]
  [0.00980264 0.00980259 0.97059214 0.00980263]
  [0.00980264 0.00980258 0.00980263 0.97059214

In [11]:
b = jnp.expand_dims(jnp.arange(xt.shape[0]), tuple(range(1, xt.ndim)))
print(b, b.shape)
q_t_teps = jnp.expand_dims(q_t_teps[b, xt], axis=-2)
print(q_t_teps, q_t_teps.shape)


[[0]
 [1]] (2, 1)
[[[[0.97059214 0.00980263 0.00980264 0.00980264]]

  [[0.00980264 0.00980261 0.97059214 0.00980263]]

  [[0.00980262 0.9705922  0.00980259 0.00980258]]

  [[0.97059214 0.00980263 0.00980264 0.00980264]]]


 [[[0.00980262 0.9705922  0.00980259 0.00980258]]

  [[0.00980264 0.00980261 0.97059214 0.00980262]]

  [[0.00980264 0.00980261 0.97059214 0.00980262]]

  [[0.97059214 0.00980263 0.00980264 0.00980264]]]] (2, 4, 1, 4)


In [ ]:
logits = qt0_y2x
log_p0t = nn.log_softmax(logits, axis=-1)
print(log_p0t, log_p0t.shape)
log_qt0 = jnp.where(qt0 <= 1e-35, -1e9, jnp.log(qt0))
print(log_qt0, log_qt0.shape)
log_qt0 = jnp.expand_dims(log_qt0, axis=list(range(1, xt.ndim)))
print(log_qt0, log_qt0.shape)
log_p0t = jnp.expand_dims(log_p0t, axis=-1)
print(log_p0t, log_p0t.shape)
log_prob = jax.nn.logsumexp(log_p0t + log_qt0, axis=-2)
print(log_prob, log_prob.shape)

In [ ]:
qt0 = uni.transition(t)
xt_onehot = jax.nn.one_hot(xt, S)
print(xt_onehot.shape)
p0t = jax.nn.softmax(logits, axis=-1)
print(p0t, p0t.shape)
qt0 = jnp.expand_dims(qt0, axis=list(range(1, xt.ndim - 1)))
print(qt0, qt0.shape)
prob_all = p0t @ qt0
print(prob_all.shape)
log_prob = jnp.log(prob_all + 1e-35)
print(log_prob, log_prob.shape)
log_xt = jnp.sum(log_prob * xt_onehot, axis=-1)
print(log_xt, log_xt.shape)

In [ ]:
qt = uni.transition(t)
b = jnp.expand_dims(jnp.arange(B), tuple(range(1, xt.ndim)))
qt0 = qt[b, xt]
print(qt0, qt0.shape)
logits = jnp.where(qt0 <= 0.0, -1e9, jnp.log(qt0))
print(logits, logits.shape)
xt = jax.random.categorical(sample_rng, logits)
print(xt, xt.shape)

In [ ]:
ll_xt = xt #B, D
ll_all =  logits# B, D, S
loss = -(
    (S - 1) * ll_xt
    + jnp.sum(utils.log1mexp(ll_all), axis=-1)
    - utils.log1mexp(ll_xt)
)
print(loss, loss.shape)

In [ ]:
ll_xt = xt #B, D
ll_all =  logits
xt_onehot = jax.nn.one_hot(xt, S)
b = jnp.expand_dims(jnp.arange(xt.shape[0]), tuple(range(1, xt.ndim)))
print(b, b.shape)
qt0_x2y = uni.transition(t)
print(qt0_x2y, qt0_x2y.shape)
qt0_y2x = jnp.transpose(qt0_x2y, (0, 2, 1))
print(qt0_x2y, qt0_x2y.shape)
qt0_y2x = qt0_y2x[b, xt]
print(qt0_x2y, qt0_x2y.shape)
ll_xt = jnp.expand_dims(ll_xt, axis=-1)
print("ll", ll_xt, ll_xt.shape)
backwd = jnp.exp(ll_all - ll_xt) * qt0_y2x
print(backwd , backwd.shape)


In [ ]:
first_term = jnp.sum(backwd * (1 - xt_onehot), axis=-1)
print(first_term , first_term.shape)
qt0_x2y = qt0_x2y[b, xt]
print(qt0_x2y, qt0_x2y.shape)
fwd = (ll_xt - ll_all) * qt0_x2y
print(fwd, fwd.shape)
second_term = jnp.sum(fwd * (1 - xt_onehot), axis=-1)
print(second_term, second_term.shape)
loss = first_term - second_term
print(loss, loss.shape)

In [ ]:
weight = jnp.ones((B, ))
weight = jnp.expand_dims(weight, axis=list(range(1, loss.ndim)))
print(weight, weight.shape)
loss = loss * weight
print(loss, loss.shape)
loss = jnp.sum(loss) / xt.shape[0]
print(loss, loss.shape)

In [ ]:
"""
The main bottleneck is the design of the conditional marginal parameterization, which requires non-trivial trade-offs between computational cost 
and flexibility of the architectures; score matching for general categorical discrete variables does not benefit from prior knowledge about ordinal 
discrete data; and finally unifying score matching between continu- ous and discrete spaces would be needed to handle data in mixed spaces
"""